# FEATURE ENGINEERING OPERATIONS - {"CUSTOMER CHURN" DATASET}

## 1. Importing Modules and Setting Configurations

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sb

from scipy import stats as sts

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectKBest, mutual_info_classif

from pickle import dump, load

import warnings
warnings.filterwarnings('ignore')

from sklearn import set_config
set_config(display='diagram')

In [2]:
# PD Options

pd.set_option('display.min_rows', 5)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 4)

In [3]:
# SB Options

sb.set_theme(context='notebook', style='whitegrid', palette='pastel', font='times new roman', font_scale=1.25)

## 2. Feature Enginerring Train, Validation, and Test Datasets

### 2.1 Prepare Target Feature

In [4]:
def prep_target(ytar):
    ytar = ytar.apply(lambda x: 1 if x=='yes' else 0)
    #ytar = ytar.astype('category')
    
    return ytar

### 2.2 Handle Oultiers

In [5]:
def outliers_detect_handle(df):
     
    fea_flo = df.select_dtypes(include='float').columns.values.tolist()
    fea_int = df.select_dtypes(include='int').columns.values.tolist()

    fea_num = fea_flo + fea_int
    fea_num
    
    # OUTLIER DETECTION CODE ----------------------------------------------------------------------------------------------
    
    for fea in fea_num:
      
        print(f'Outlier Detection for Feature : {fea} ------------------------------------------------------------------- \n')

        mn = df[fea].min()
        mx = df[fea].max()
        #print(f'Minimum Value : {mn} and Maximum Value : {mx} \n')

        q1,q3 = df[fea].quantile([0.25,0.75])         #  for 1st and 3rd quartile
        #print(f'1st Quartile : {q1} and 3rd Quartile : {q3} \n')

        lb = round(q1 - (q3-q1)*1.5,4)
        ub = round(q3 + (q3-q1)*1.5,4)
        #print(f'Lower Bound : {lb} and Higher Bound : {ub} \n')

        filtl = df[fea] < lb
        out_low = df[filtl]
        #print(f'No. of Outliers below Lower Bound ({lb}) are : {out_low.shape[0]}')

        filtu = df[fea] > ub
        out_high = df[filtu]
        #print(f'No. of Outliers above Upper Bound ({ub}) are : {out_high.shape[0]}')

        out_df = df[(filtl | filtu)]
        print(f' --> Total No. of Outliers before : {out_df.shape[0]} \n')

        per = (out_df.shape[0]/df[fea].shape[0])*100
        #print(f'Percentage of Outliers Records are : {round(per,4)} % \n\n')

#         plt.figure(figsize=(15,5))
#         plt.subplot(2,2,1)
#         plt.title('KDE Plot Before')
#         sb.kdeplot(data=df, x=fea)

#         plt.subplot(2,2,2)
#         plt.title('BOX Plot Before')
#         sb.boxplot(data=df, x=fea)

        # OUTLIER HANDLING USING CAPPING TECHNIQUE -----------------------------------------------------------------------------
        df.loc[filtl,fea] = lb                        
        df.loc[filtu,fea] = ub

        # OUTLIER DETECTION CODE -----------------------------------------------------------------------------------------------
        filtl = df[fea] < lb
        out_low = df[filtl]
        # print(f'No. of Outliers below Lower Bound ({lb}) are : {out_low.shape[0]}')

        filtu = df[fea] > ub
        out_high = df[filtu]
        # print(f'No. of Outliers above Upper Bound ({ub}) are : {out_high.shape[0]}')

        out_df = df[(filtl | filtu)]
        print(f' --> Total No. of Outliers after : {out_df.shape[0]} \n')

        per = (out_df.shape[0]/df[fea].shape[0])*100
        #print(f'Percentage of Outliers Records are : {round(per,4)} %')

#         plt.subplot(2,2,3)
#         plt.title('KDE Plot After')
#         sb.kdeplot(data=df, x=fea)

#         plt.subplot(2,2,4)
#         plt.title('BOX Plot After')
#         sb.boxplot(data=df, x=fea)

#         plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.8)
#         plt.show()
   
    return df

### 2.3 Feature Transformations

In [6]:
# cols=[1,5,7,10,13,15,16,17,18]

num_ft_pt = ColumnTransformer([
    ('yj',PowerTransformer(method='yeo-johnson', standardize=False),[1,5,7,10,13,15,16,17,18])
    ], remainder='passthrough')

### 2.4 Scaling

In [7]:
num_scl_ss = ColumnTransformer([
                ('ss',StandardScaler(),[0,1,2,3,4,5,6,7,8,13,14,15,16,17,18])
                ],
                remainder='passthrough')

### 2.5 Categorical Feature Encoding

In [8]:
cat_enc_ohe = ColumnTransformer([
                ('ohe',OneHotEncoder(drop='first', sparse_output=False, dtype='int8'),[15,16,17,18])
                ],
                remainder='passthrough')

In [9]:
steps1 = [('pt',num_ft_pt),
     ('ss',num_scl_ss),
     ('ohe',cat_enc_ohe)
     ]

pipe1 = Pipeline(steps1)            # for training dataset

### 2.6 Imbalance Dataset Handling

In [10]:
sm = SMOTE(random_state=46)

### 2.7 Feature Selection Technique

In [11]:
skb = SelectKBest(mutual_info_classif, k='all')

## 3 Training Data

### 3.1 Train Dataset

In [12]:
tr = pd.read_pickle('cc_train_pp.pkl')


print(f'Shape of the train dataset : {tr.shape}')
tr.head(5)

Shape of the train dataset : (4050, 20)


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
3199,NJ,165,area_code_415,no,yes,17,177.9,68,30.24,153.9,112,13.08,251.2,121,11.30,10.6,7,2.86,1,no
4073,MN,136,area_code_415,no,no,0,221.4,120,37.64,249.5,78,21.21,192.0,68,8.64,13.6,2,3.67,0,no
2060,MT,97,area_code_415,no,yes,15,117.6,97,19.99,196.3,126,16.69,157.4,113,7.08,6.4,3,1.73,1,no
1634,DE,81,area_code_510,yes,no,0,250.6,85,42.60,187.9,50,15.97,120.3,131,5.41,7.8,5,2.11,1,no
3687,OK,28,area_code_408,no,no,0,225.7,70,38.37,206.9,90,17.59,167.9,88,7.56,13.8,3,3.73,1,no


In [13]:
Xtr = tr.drop(columns='churn')
ytr = tr['churn']

### 3.2 FE Steps

In [14]:
ytr = prep_target(ytr)

In [15]:
Xtr = outliers_detect_handle(Xtr)

Outlier Detection for Feature : total_day_minutes ------------------------------------------------------------------- 

 --> Total No. of Outliers before : 22 

 --> Total No. of Outliers after : 0 

Outlier Detection for Feature : total_day_charge ------------------------------------------------------------------- 

 --> Total No. of Outliers before : 22 

 --> Total No. of Outliers after : 0 

Outlier Detection for Feature : total_eve_minutes ------------------------------------------------------------------- 

 --> Total No. of Outliers before : 32 

 --> Total No. of Outliers after : 0 

Outlier Detection for Feature : total_eve_charge ------------------------------------------------------------------- 

 --> Total No. of Outliers before : 33 

 --> Total No. of Outliers after : 0 

Outlier Detection for Feature : total_night_minutes ------------------------------------------------------------------- 

 --> Total No. of Outliers before : 35 

 --> Total No. of Outliers after : 0 



In [16]:
Xtr = pipe1.fit_transform(Xtr,ytr)

In [17]:
Xtr, ytr = sm.fit_resample(Xtr,ytr)

In [18]:
Xtr = skb.fit_transform(Xtr, ytr)
print(skb.get_support())

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


In [19]:
Xtr.shape

(6960, 69)

In [20]:
Xtr[:1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  1.59736629,
         1.64273249, -1.61910902,  0.60083031,  1.05438831,  0.10655575,
         1.14650785,  0.10238964, -0.30343896, -0.04517626, -0.04556432,
        -0.9277831 , -0.9282404 ,  1.01737037,  1.0

### 3.3 Save the train FE data as CSV, PKL file

In [21]:
Xtr = pd.DataFrame(Xtr)                # ndarray to df

In [22]:
tr = pd.concat([Xtr,ytr], axis=1)   # concat feature df and target series

In [23]:
tr.shape

(6960, 70)

In [24]:
tr.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,churn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.1465,0.1024,-0.3034,-0.0452,-0.0456,-0.9278,-0.9282,1.0174,1.0155,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.1349,1.2493,-1.5019,0.7618,0.7619,0.9893,0.9899,-0.1726,-0.1727,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.5194,-1.4436,-0.3034,-1.1638,-1.1640,-0.0775,-0.0765,-0.8682,-0.8695,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.4219,-0.9316,-0.3034,1.3034,1.3031,-0.2460,-0.2464,-1.6139,-1.6154,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.5194,1.3354,-0.3034,0.8415,0.8416,0.1350,0.1358,-0.6571,-0.6551,0


In [25]:
tr.to_csv('cc_train_fe.csv', index=False)
tr.to_pickle('cc_train_fe.pkl')

print('Train dataset saved successfully in CSV and PKL files ...')

Train dataset saved successfully in CSV and PKL files ...


## 4 Validation Data FE

### 4.1 Validation Dataset

In [26]:
val = pd.read_pickle('cc_valid_pp.pkl')

print(f'Shape of the validation dataset : {val.shape}')
val.head(5)

Shape of the validation dataset : (100, 20)


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
2487,KY,96,area_code_415,no,yes,40,108.6,90,18.46,206.4,154,17.54,126.3,118,5.68,13.4,4,3.62,0,no
2711,WA,152,area_code_510,no,no,0,161.4,84,27.44,163.6,88,13.91,153.2,121,6.89,11.8,5,3.19,1,no
4237,TN,74,area_code_510,no,no,0,159.4,86,27.10,210.0,122,17.85,172.4,99,7.76,8.5,2,2.30,1,no
2673,AZ,94,area_code_415,no,no,0,220.8,111,37.54,156.2,67,13.28,187.9,89,8.46,10.5,4,2.84,2,no
1558,NH,120,area_code_510,no,yes,43,177.9,117,30.24,175.1,70,14.88,161.3,117,7.26,11.5,4,3.11,1,no


In [27]:
Xval = val.drop(columns='churn')
yval = val['churn']

### 4.2 FE Steps

In [28]:
yval = prep_target(yval)

In [29]:
steps2 = [('pp1',pipe1),
     ('sb',skb)
     ]

pipe2 = Pipeline(steps2)

dump(pipe2, open('cc_pipe2_val_te.pkl','wb'))
print('"pipe2" Features Saved Successfully')

"pipe2" Features Saved Successfully


In [30]:
Xval = pipe2.transform(Xval)
Xval.shape

(100, 69)

In [31]:
Xval[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, -0.07908712743963611,
        1.7149019358403352, -0.5023471426321819, 2.6981732302547425,
        0.9058597295559861, 1.174946280105122, -0.012493308385580418,
        1.1776873310794866, -1.5018581756320228, -1.3307018822315966,
        -1.3309784937370601, 0.12500682750835182, 0.12401624975327542,
        -1.493324161700367, -1.4948207486786635]], dtype=object)

In [32]:
Xval=Xval.astype('float')

### 4.3 Save the validation FE data as CSV, PKL file

In [33]:
yval = yval.values
yval = yval.reshape((100,1))
yval[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [34]:
val = np.concatenate([Xval,yval], axis=1)         

In [35]:
val = pd.DataFrame(val)   # array to df
val.rename(columns={69:'churn'}, inplace=True)
val.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,churn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0125,1.1777,-1.5019,-1.3307,-1.3310,0.1250,0.1240,-1.4933,-1.4948,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4219,0.5662,-0.3034,-0.3513,-0.3511,-0.7333,-0.7324,-0.9526,-0.9544,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.1349,-0.6725,-0.3034,-0.3884,-0.3882,0.1972,0.1972,-0.5666,-0.5658,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0125,0.0744,0.5730,0.7506,0.7510,-0.8817,-0.8811,-0.2551,-0.2531,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0125,0.4533,-0.3034,-0.0452,-0.0456,-0.5027,-0.5036,-0.7898,-0.7891,0.0


In [36]:
val.shape

(100, 70)

In [37]:
val.to_csv('cc_valid_fe.csv', index=False)
val.to_pickle('cc_valid_fe.pkl')

print('Validation dataset saved successfully in CSV and PKL files ...')

Validation dataset saved successfully in CSV and PKL files ...


## 5. Test Dataset FE

### 5.1 Test Dataset

In [38]:
te = pd.read_pickle('cc_test_pp.pkl')

print(f'Shape of the test dataset : {te.shape}')
te.head(5)

Shape of the test dataset : (100, 20)


,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
4221,GA,122,area_code_408,no,no,0,93.3,109,15.86,214.4,104,18.22,176.7,104,7.95,12.1,2,3.27,0,no
1476,MO,80,area_code_408,yes,yes,15,159.3,110,27.08,170.6,120,14.50,141.2,82,6.35,11.9,5,3.21,1,no
904,NC,64,area_code_408,no,yes,19,291.1,150,49.49,226.7,123,19.27,219.1,67,9.86,7.5,2,2.03,1,no
3527,NY,119,area_code_408,no,no,0,153.2,131,26.04,116.6,85,9.91,218.5,74,9.83,10.1,5,2.73,1,no
3656,OH,82,area_code_415,no,no,0,207.2,114,35.22,163.7,97,13.91,192.8,79,8.68,9.3,4,2.51,7,no


In [39]:
Xte = te.drop(columns='churn')
yte = te['churn']

### 5.2 FE Steps

In [40]:
yte = prep_target(yte)

In [41]:
Xte = pipe2.transform(Xte)
Xte.shape

(100, 69)

In [42]:
Xte[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5654108541600653,
        -0.5926784100938677, 0.46591572738994985, 0.1984529532036502,
        0.20979268030826703, 0.6760492984372589, -1.1348639665891906,
        0.6793344892577267, -1.5018581756320228, -1.6145192264084471,
        -1.6146862059414007, 0.28543195906049273, 0.28444999949311073,
        -0.4802016291235067, -0.48088478608696056]], dtype=object)

In [43]:
Xte=Xte.astype('float')

### 5.3 Save the test FE data as CSV, PKL file

In [44]:
yte = yte.values
yte = yte.reshape((100,1))
yte[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [45]:
te = np.concatenate([Xte,yte], axis=1)   

In [46]:
te = pd.DataFrame(te)   # array to df
te.rename(columns={69:'churn'}, inplace=True)
te.head()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,churn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.1349,0.6793,-1.5019,-1.6145,-1.6147,0.2854,0.2844,-0.4802,-0.4809,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4219,0.5944,-0.3034,-0.3902,-0.3904,-0.5929,-0.5932,-1.1938,-1.1956,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.1349,-1.0401,-0.3034,2.0547,2.0550,0.5321,0.5322,0.3721,0.3723,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.4219,-0.0789,-0.3034,-0.5034,-0.5039,-1.6758,-1.6761,0.3600,0.3589,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0125,-0.3838,3.4065,0.4983,0.4978,-0.7313,-0.7324,-0.1566,-0.1548,0.0


In [47]:
te.shape

(100, 70)

In [48]:
te.to_csv('cc_test_fe.csv', index=False)
te.to_pickle('cc_test_fe.pkl')

print('Test dataset saved successfully in CSV and PKL files ...')

Test dataset saved successfully in CSV and PKL files ...
